In [1]:
import os
import json
import logging
from gscv_configs import gscv_cfg_dt, gscv_cfg_gb, gscv_cfg_knn, gscv_cfg_mlp, gscv_cfg_rf, gscv_cfg_sv, gscv_cfg_xgb, gscv_cfg_nb, gscv_cfg_lr
from sklearn.metrics import f1_score, accuracy_score
from utils import retrieve_latest_train_test
from prettytable import PrettyTable
import random
import pandas as pd

cfgs = [gscv_cfg_dt, gscv_cfg_gb, gscv_cfg_knn, gscv_cfg_mlp, gscv_cfg_rf, gscv_cfg_xgb, gscv_cfg_nb, gscv_cfg_lr, gscv_cfg_sv]

train, test = retrieve_latest_train_test()#drop_col=["Age_true", "AgeGroup", "Embarked_S", "Title_Noble"])

# TRAIN
X_train = train.drop(["Survived"], axis=1)
y_train=train["Survived"]

# TEST
X_test=test.drop(["Survived"], axis=1)
y_test=test["Survived"]

output = PrettyTable()
output.field_names = ["Name", "Estimator", "F1", "Accuracy"]
for cfg in cfgs:
    dir = f"../gscv/{cfg.name}"
    files = sorted([f for f in os.listdir(dir) if (f.endswith(".json"))], reverse=True)
    try:
        file = files[0]
        with open(f"{dir}/{file}") as f:
            gscv = json.load(f)

        random.seed(10)

        estimator = cfg.estimator
        param_grid = gscv['best_params_']

        estimator.set_params(**param_grid)
        estimator.random_state = 500
        estimator.fit(X_train, y_train)

        yhat = estimator.predict(X_test)

        output.add_row([cfg.name, estimator, round(f1_score(yhat, y_test), 4), round(accuracy_score(yhat, y_test), 4)])

        KAGGLE_TRAIN = pd.read_csv(f"../data/preprocessed_20221117.csv", index_col=0)
        # KAGGLE_TRAIN = KAGGLE_TRAIN.drop(["Age_true", "AgeGroup", "Embarked_S", "Title_Noble"], axis=1)
        KAGGLE_TEST_X = pd.read_csv(f"../data/test_preprocessed_20221125.csv", index_col=0)
        # KAGGLE_TEST_X = KAGGLE_TEST_X.drop(["Age_true", "AgeGroup", "Embarked_S", "Title_Noble"], axis=1)
        KAGLLE_TEST_INDEX = pd.read_csv(f"../data/test.csv", index_col=0).index

        estimator.set_params(**param_grid)
        estimator.fit(KAGGLE_TRAIN.drop(["Survived"], axis=1), KAGGLE_TRAIN["Survived"])

        yhat = estimator.predict(KAGGLE_TEST_X)

        subm = pd.DataFrame(yhat, columns=["Survived"], index=KAGLLE_TEST_INDEX)
        subm.index.name="PassengerId"

        subm.to_csv(f"../submission/{cfg.name}_submission.csv")

    except IndexError:
        logging.warning(f"There are no results for {cfg.name}")

output.sortby = 'Accuracy'
print(output)

/Users/mariusj/Documents/Edu/UMA/lectures/Data Mining/Project/dm1-titanic/.venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:455: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(


+-----------------------+---------------------------------------------------------------------------------+--------+----------+
|          Name         |                                    Estimator                                    |   F1   | Accuracy |
+-----------------------+---------------------------------------------------------------------------------+--------+----------+
|   gradient_boosting   | GradientBoostingClassifier(learning_rate=0.3, max_depth=8, max_features='log2', | 0.7407 |  0.791   |
|                       |                                     n_estimators=25, random_state=500)          |        |          |
|   knearest_neighbor   | KNeighborsClassifier(algorithm='ball_tree', n_neighbors=10, weights='distance') | 0.7465 |  0.7948  |
|      naive_bayes      |                                   GaussianNB()                                  | 0.7692 |  0.7985  |
|     random_forest     |  RandomForestClassifier(bootstrap=False, max_features=None, min_samples_leaf=3